# **THE TRUTH IS OUT THERE** 

Datos: https://www.kaggle.com/NUFORC/ufo-sightings<br>
Es un registro de avistamientos de OVNI llevado a cabo por National UFO Reporting Center (NUFORC)<br>
Contiene registros de avistamientos desde 1906 hasta 2014<br>
En este notebook haremos una pequeña preparación de datos para dejar un set limpio para analizar luego con visualizaciones. Por ese motivo no va a ser un notebook muy entretenido. Las visualizaciones las hare en otro notebook

Consideraciones:
* voy a estar utilizando algunas funciones de la libreria funpymodeling, que nos ayuda sobretodo con el analisis exploratorio y la preparación de datos https://pypi.org/project/funpymodeling/
* Tambien voy a usar la libreria reverse_geocoding para encontrar nombres de países a partir de sus coordenadas https://github.com/thampiman/reverse-geocoder
* Este notebook no fue realizado por un experto. Seguramente haya mejores maneras de afrontar las mismas problematicas. Quedo atento a cualquier comentario o sugerencia 

![The kaggle logo][1]
[1]: https://cdn.forbes.com.mx/2015/03/Internet-XFiles-Mulder.jpg

# **Comencemos**

**Importamos librerias que vamos a utilizar y cargamos los datos en un dataframe**

In [1]:
pip install funpymodeling


     |████████████████████████████████| 215 kB 1.7 MB/s 
     |████████████████████████████████| 11.6 MB 6.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.1
    Uninstalling matplotlib-3.2.1:
      Successfully uninstalled matplotlib-3.2.1
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.10.0
    Uninstalling seaborn-0.10.0:
      Successfully uninstalled seaborn-0.10.0
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install reverse_geocoder

     |████████████████████████████████| 2.2 MB 1.4 MB/s 
  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268088 sha256=a574fecbf681be55d4d674c807e835a298970ca3edea04c1da8872f81f3af988
  Stored in directory: /root/.cache/pip/wheels/34/6e/70/5423639428a2cac8ea7eb467214a4254b549b381f306a9c790
Successfully built reverse-geocoder
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
from qgrid import show_grid
from pandas_profiling import ProfileReport
import reverse_geocoder as rg

In [4]:
# Cargamos los datos
data = pd.read_csv("../input/ufo-sightings/scrubbed.csv", sep=",")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Con la función status de funpymodeling, verificamos el estado de salud de los datos
# tipos de datos: excepto por longitude, todos vienen en object. Llama la atención
# hay muchisimos nulos en state, country y shape. Tambien tenemos algunos en comments
# tenemos dos columnas que muestran el tiempo que duro el avistamiento. Nos quedaremos con la que tenga más sentido
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,datetime,0,0.000000,0,0.0,69586,object
1,city,0,0.000000,0,0.0,19900,object
2,state,5797,0.072163,0,0.0,67,object
3,country,9670,0.120375,0,0.0,5,object
4,shape,1932,0.024050,0,0.0,29,object
5,duration (seconds),0,0.000000,0,0.0,706,object
6,duration (hours/min),0,0.000000,0,0.0,8349,object
7,comments,15,0.000187,0,0.0,79997,object
8,date posted,0,0.000000,0,0.0,317,object
9,latitude,0,0.000000,0,0.0,23312,object


In [6]:
# Veamos algunos datos
data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


INDICE

CONVERSIÓN DE TIPO DE DATOS
* 1° OBSERVACIÓN: 'DATETIME' TIENE COMO TIPO DE DATOS: OBJECT
* 2° OBSERVACIÓN. 'LATITUDE' TIENE COMO TIPO DE DATOS: OBJECT
* 3° OBSERVACIÓN. TENEMOS DOS COLUMNAS PARA EL TIEMPO DE AVISTAMIENTO. AMBAS FIGURAN COMO OBJECT
* 4° OBSERVACIÓN. DATE POSTED FIGURA COMO OBJECT
TRATAMIENTO DE NULOS
* 5° OBSERVACIÓN: NULOS EN COUNTRY, STATE y CITY
* 6° OBSERVACIÓN: NULOS EN SHAPE Y COMMENTS
OTRA TRANSFORMACIONES
* 7° OBSERVACIÓN: CREAR NUEVA COLUMNA DE TIEMPO DE AVISTAMIENTO (MINUTOS)
* ULTIMOS PASOS
* (EDIT) 8° OBSERVACIÓN: ASIGNAR NOMBRE AMIGABLE A LOS PAISES

ALGUNAS ACLARACIONES

* No voy a estar reemplazando las variables originales, les asignare otro nombre y las eliminare al final cuando este seguro de que me quedo como quiero

****************************************************************************************

![The kaggle logo][2]
[2]: https://i.ytimg.com/vi/FtyEo-8OWNA/hqdefault.jpg

# **CONVERSION DE TIPO DE DATOS**

#1° OBSERVACIÓN: 'DATETIME' TIENE COMO TIPO DE DATOS: OBJECT

In [7]:
# Enfoque: convertirlo a datetime64
data['datetime_dat']=data['datetime'].astype('datetime64')
# No permite hacer la conversion porque aparentemente algunas fechas tienen la hora como '24:00'
# entonces vamos a reemplazarlo por '00:00'


ParserError: hour must be in 0..23: 10/11/2006 24:00

In [8]:
# verifico si efectivamente hay registros con la hora cargada como '24:00'
data[data['datetime'].str.contains("24:00")]
# hay 694 regirstros con hora '24:00'

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
388,10/11/2006 24:00,rome,ny,us,oval,120,a min or two,I was walking from the garage to the house&#44...,2/1/2007,43.2127778,-75.456111
693,10/1/2001 24:00,chulucanas-piura la vieja (peru),NaN,NaN,other,6312000,2 years,go to: http://www.24horas.com.pe/data/videos/...,3/4/2003,-5.129547,-80.120569
962,10/1/2012 24:00,novi,mi,us,triangle,300,5 minutes,V shaped and 8 big and very brite lights&#44mo...,10/30/2012,42.4805556,-83.475556
1067,10/12/2003 24:00,salatiga (indonesia),NaN,NaN,disk,22,22 seconds,UFO in Salatiga&#44Indonesia,10/31/2003,-7.33683,110.498817
1221,10/12/2013 24:00,cincinnati,oh,us,fireball,300,3-5 minutes,A bright orange light split into four&#44 did ...,10/14/2013,39.1619444,-84.456944
...,...,...,...,...,...,...,...,...,...,...,...
79136,9/4/2005 24:00,boonville,nc,us,cigar,10800,3+ hours,Unusual activity in our small community&#44 cr...,9/9/2013,36.2325,-80.708333
79137,9/4/2005 24:00,redondo beach,ca,us,triangle,300,5 minutes,Triangulat UFO seen over Redondo Beach&#44 Cal...,12/16/2005,33.8492,-118.387500
79740,9/7/2002 24:00,portsmouth,oh,us,changing,900,15 minutes,A bright red light hoovering in the sky.,9/13/2002,38.7317,-82.997778
79759,9/7/2004 24:00,montrose,co,us,unknown,1200,20 minutes,Stationary object with three pulsating lights ...,9/9/2004,38.4783,-107.875556


In [9]:
# donde encuentra un '24:00' le asigna '00:00'
data['datetime_dat'] = data['datetime'].replace({'24:00': '00:00'}, regex=True)

In [10]:
# Probemos ahora la conversión al tipo de datos datetime
data['datetime_dat']=data['datetime_dat'].astype('datetime64')

**********************

#2° OBSERVACIÓN: 'LATITUDE' TIENE COMO TIPO DE DATOS: OBJECT

In [11]:
# Enfoque: convertirlo a float64
data['latitude_num']=data['latitude'].astype('float64')
# no lo permite porque encontró un registro donde aparece una letra 'q'. 
# desconozco si es algo propio de la codificacion de las latitudes. Pero como es solo un registro, lo elimino 

ValueError: could not convert string to float: '33q.200088'

In [12]:
# con freq_tbl de funpymodeling puedo verificar la frecuencia del dataframe
# veamos si efectivamente encuentro algun caracter no numerico
freq_tbl(data.latitude.str.contains('[A-Za-z]'))
#efectivamente, encontró uno

,latitude,frequency,percentage,cumulative_perc
0,False,65535,0.815802,0.999985
1,True,1,0.000012,1.000000


In [13]:
#veamos qué registro es
data[data['latitude'].str.contains('[A-Za-z]', na=False)]  
#es el registro 43782. Vemos que en latitud aparece una 'q' en el medio

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_dat
43782,5/22/1974 05:30,mescalero indian reservation,nm,NaN,rectangle,180,two hours,Huge rectangular object emmitting intense whit...,4/18/2012,33q.200088,-105.624152,1974-05-22 05:30:00


In [14]:
# Vemos que es el 43782. Vamos a eliminarlo con su índice 
data= data.drop([43782])

In [15]:
# MUY IMPORTANTE
# en este punto, es recomendable reiniciar el índice de la tabla, 
# ya que luego obtendremos los países faltantes a traves de reverse_geocoding.
data = data.reset_index(drop=True)

In [16]:
# probemos otra vez la conversión de object a numerico
data['latitude_num']=data['latitude'].astype('float64')
# ahora si nos permitió

**********************

#3° OBSERVACIÓN. TENEMOS DOS COLUMNAS PARA EL TIEMPO DE AVISTAMIENTO. AMBAS FIGURAN COMO OBJECT

In [17]:
# Enfoque: eliminar la que muestra en minutos, ya que tiene muchisimos caracteres no numericos (lo haremos al final del notebook).
# me quedo con la que muestra en segundos (probablemente al final la transformemos a minutos para facilitar el analisis)
# entonces, tomamos 'duration (seconds)' y la transformamos a float64
data['duration(seconds)_num']=data['duration (seconds)'].astype('float64')
# No lo puede converir porque encuentra valores como '2`', es decir tienen caracteres no numericos

ValueError: could not convert string to float: '2`'

In [18]:
# veamos cuáles son esos registros que traen apostrofe
data[data['duration (seconds)'].str.contains("`",na=False)]
# son 3, podriamos borrarlos. O mejor aun, probemos otra cosa
# vamos a editarle el valor, como hicimos en la 1°observación (reemplazar '24:00' por '00:00')

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_dat,latitude_num
27822,2/2/2000 19:33,bouse,az,us,NaN,2`,each a few seconds,Driving through Plomosa Pass towards Bouse Loo...,2/16/2000,33.9325000,-114.005000,2000-02-02 19:33:00,33.932500
35692,4/10/2005 22:52,santa cruz,ca,us,NaN,8`,eight seconds,2 red lights moving together and apart with a ...,4/16/2005,36.9741667,-122.029722,2005-04-10 22:52:00,36.974167
58590,7/21/2006 13:00,ibague (colombia),NaN,NaN,circle,0.5`,1/2 segundo,Viajaba a 27.000 pies en un avion comercial ve...,10/30/2006,4.440663,-75.244141,2006-07-21 13:00:00,4.440663


In [19]:
# le asigno a cada combinacion de fila/columna, el valor sin apostrofe.
data.loc[27822,'duration (seconds)'] = 2
data.loc[35692,'duration (seconds)'] = 8
data.loc[58590,'duration (seconds)'] = 0.5

In [20]:
# ahora si, procedemos a convertir el tipo de datos a float64

data['duration(seconds)_num']=data['duration (seconds)'].astype('float64')

**********************

#4° OBSERVACIÓN. DATE POSTED FIGURA COMO OBJECT

In [21]:
# Enfoque: convertirlo a datetime64
data['datePosted_dat']=data['date posted'].astype('datetime64')
#este no dio problemas. Ya tenemos la conversión

Terminamos por el momento con las conversiones de datos. Proximo paso: tratamiento de nulos

****************************************************************************************

![The kaggle logo][3]
[3]: https://elcomercio.pe/resizer/CMBy5GAqgqFYqvWBHQFqMM58u7A=/580x330/smart/filters:format(jpeg):quality(75)/cloudfront-us-east-1.images.arcpublishing.com/elcomercio/3RKS3OTR4BFUBIXJOKYADRE4AI.jpg

#TRATAMIENTO DE NULOS

Veamos como estan nuestros datos

In [22]:
status(data)
#recordemos que estamos duplicando las columnas, al final eliminaremos las originales que ya tienen su "arreglo"
#vemos que hay nulos en state, country, shape y comments

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,datetime,0,0.000000,0,0.0,69585,object
1,city,0,0.000000,0,0.0,19899,object
2,state,5797,0.072164,0,0.0,67,object
3,country,9669,0.120364,0,0.0,5,object
4,shape,1932,0.024050,0,0.0,29,object
5,duration (seconds),0,0.000000,0,0.0,703,object
6,duration (hours/min),0,0.000000,0,0.0,8349,object
7,comments,15,0.000187,0,0.0,79996,object
8,date posted,0,0.000000,0,0.0,317,object
9,latitude,0,0.000000,0,0.0,23311,object


#5° OBSERVACIÓN: NULOS EN COUNTRY, STATE y CITY

In [23]:
# Enfoque: hay 9669 nulos en country. Podriamos eliminarlos...
# ..Pero si yo tengo correctamente las coordenadas, podría obtener su país a traves de ellas
# esto se logra gracias a reverse_geocoding

In [24]:
# creo una variable con la tupla de lat y long (para pasarle a reverse_geocoding y haga su magia)
coordinates = list(zip(data['latitude_num'], data['longitude ']))

In [25]:
import reverse_geocoder as rg

In [26]:
# le paso al geocoding, mis coordenadas para que me devuelva países
# lo hace en una lista
results = rg.search(coordinates) # default mode = 2
# print(results)

Loading formatted geocoded file...


In [27]:
# veamos que tiene la lista "results" para el primer registro
#[('lat', '29.88327'), ('lon', '-97.94139'), ('name', 'San Marcos'), ('admin1', 'Texas'), ('admin2', 'Hays County'), ('cc', 'US')])

In [28]:
# ahora veamos que tiene el primer registro de nuestros datos
data.loc[0]
# los datos coinciden. Este avistamiento fue en San Marcos, Texas (Estados unidos)
# latitud y longitud vemos que es la misma practicamente (es lo que usa para encontrar países)
# a partir de ahi obtuve su país (aparece como cc), y tambien su estado(admin1) y ciudad(name)

datetime                                                  10/10/1949 20:30
city                                                            san marcos
state                                                                   tx
country                                                                 us
shape                                                             cylinder
duration (seconds)                                                    2700
duration (hours/min)                                            45 minutes
comments                 This event took place in early fall around 194...
date posted                                                      4/27/2004
latitude                                                        29.8830556
longitude                                                         -97.9411
datetime_dat                                           1949-10-10 20:30:00
latitude_num                                                       29.8831
duration(seconds)_num    

In [29]:
# ¿Qué hacemos ahora?
# ahora unimos nuestra data con lo que nos interesa de la lista. Primero convertimos esa lista en un dataframe
results_df = pd.DataFrame(results)


In [30]:
# ahora simplemente seleccionamos que columnas queremos agregar de results_df a nuestros datos
# ¿se acuerdan que reiniciamos el indice cuando eliminamos un registro en el 2° problema? Fue para hacer esta union facilmente

data['country_c']=results_df['cc']
data['city_c']=results_df['name']
data['state_c'] = results_df['admin1']

In [31]:
# et voilà, tenemos data de todos esos paises que eran aparecian nulos, a partir de sus coordenadas
(data[['country','country_c','state','state_c','city','city_c']])
# por ejemplo para el registro 1, venia país nulo. Sin embargo ahora sabemos que es de US. Lo confirma que el estado es Texas

,country,country_c,state,state_c,city,city_c
0,us,US,tx,Texas,san marcos,San Marcos
1,NaN,US,tx,Texas,lackland afb,Lackland Air Force Base
2,gb,GB,NaN,England,chester (uk/england),Blacon
3,us,US,tx,Texas,edna,Edna
4,us,US,hi,Hawaii,kaneohe,Kane'ohe
...,...,...,...,...,...,...
80326,us,US,tn,Tennessee,nashville,Nashville
80327,us,US,id,Idaho,boise,Boise
80328,us,US,ca,California,napa,Napa
80329,us,US,va,Virginia,vienna,Vienna


In [32]:
import numpy as np

In [33]:
# Nota: hay algunos estados en esta nueva columna state_c que vienen vacios, a pesar de que los obtuvimos con geocoding
# vamos a reemplazarlos por el mismo valor que aparezca en ciudad

# primero los pasamos a nulo
data['state_c'] = data['state_c'].replace({'': np.nan})

In [34]:
# y ahora a esos nulos le asignamos el nombre de la ciudad
data['state_c'] = data['state_c'].fillna(data['city_c'])

****************

#6° OBSERVACIÓN: NULOS EN SHAPE Y COMMENTS

In [35]:
# Enfoque: aca si no podemos hacer nada por estos nulos, solo le asignaremos un string 'No data'
data['shape'] = data['shape'].fillna("No data")
data['comments'] = data['comments'].fillna("No data")

***********************************************************************

#OTRAS TRANSFORMACIONES

#7° OBSERVACIÓN: CREAR NUEVA COLUMNA DE TIEMPO DE AVISTAMIENTO (MINUTOS)

In [36]:
# vamos a crear nuevamente la columna de tiempo de avistamiento en minutos,
# pero lo haremos a partir de la columna de segundos, que tiene datos consistentes
data['duration(minutes)_num']=data['duration(seconds)_num'] /60

#ULTIMOS PASOS

Eliminamos todas esas columnas originales que fueron tratadas

In [37]:
data=data.drop(['datetime','latitude','duration (seconds)','date posted','country','state','city','duration(seconds)_num'],axis=1)
# elimino  tambien duration (hours/min) 
data=data.drop(['duration (hours/min)'],axis=1)

Cambiamos los nombres de las nuevas columnas, y las reordenamos

In [38]:
data = data.rename(columns={"datetime_dat": "datetime", 
                             "latitude_num": "latitude", 
                             'datePosted_dat' : 'datePosted',
                             'country_c':'country',
                             'state_c':'state',
                             'city_c': 'city',
                             'duration(minutes)_num': 'duration(minutes)',
                             'longitude ': 'longitude' })

data = data[["datetime", "country", "state","city","shape","duration(minutes)","comments", "datePosted","latitude","longitude"]]

In [39]:
# Y tenemos nuestro set de datos un poco más ordenado 
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,datetime,0,0.0,0,0.0,69433,datetime64[ns]
1,country,0,0.0,0,0.0,164,object
2,state,0,0.0,0,0.0,840,object
3,city,0,0.0,0,0.0,10676,object
4,shape,0,0.0,0,0.0,30,object
5,duration(minutes),0,0.0,0,0.0,533,float64
6,comments,0,0.0,0,0.0,79997,object
7,datePosted,0,0.0,0,0.0,317,datetime64[ns]
8,latitude,0,0.0,0,0.0,19907,float64
9,longitude,0,0.0,0,0.0,19454,float64


*************************

#(EDIT) 8° OBSERVACIÓN: ASIGNAR NOMBRE AMIGABLE A LOS PAISES

Mientras revisaba los ultimos detalles este notebook, se me ocurrio que para visualizar, estaria bueno tener los nombres de los países<br>
En teoría es simple... seria buscar una tabla de países y hacer el join. Veremos que sale

<img src="https://thumbs.gfycat.com/DelightfulDisfiguredHamadryad-size_restricted.gif">

In [40]:
# Conseguí esta tabla googleando unos minutos
countryNames = pd.read_csv('../input/country-code-and-names/Comtrade Country Code and ISO list v1.2.csv', sep =';')

In [41]:
# Me quedo con las unicas columnas que me van a interesar
countryNames = countryNames.drop(['Country Code','Country Name, Full ','Country Comments','ISO3-digit Alpha','Start Valid Year','End Valid Year'],axis=1)

In [42]:
# Asi quedó mi tabla de países
countryNames

,"Country Name, Abbreviation",ISO2-digit Alpha
0,Afghanistan,AF
1,Albania,AL
2,Algeria,DZ
3,American Samoa,AS
4,Andorra,AD
...,...,...
230,Western Sahara,EH
231,World,WL
232,Yemen,YE
233,Zambia,ZM


In [43]:
# Hacemos el join (left)
data_final = data.merge(countryNames, left_on='country', right_on='ISO2-digit Alpha', how='left')

In [44]:
# Y vemos otra vez el status (...God bless status...)
status(data_final)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,datetime,0,0.000000,0,0.0,69433,datetime64[ns]
1,country,0,0.000000,0,0.0,164,object
2,state,0,0.000000,0,0.0,840,object
3,city,0,0.000000,0,0.0,10676,object
4,shape,0,0.000000,0,0.0,30,object
5,duration(minutes),0,0.000000,0,0.0,533,float64
6,comments,0,0.000000,0,0.0,79997,object
7,datePosted,0,0.000000,0,0.0,317,datetime64[ns]
8,latitude,0,0.000000,0,0.0,19907,float64
9,longitude,0,0.000000,0,0.0,19454,float64


In [45]:
# todavia hay nulos en Country Name, Abbreviation
# probablemente este sucediendo porque hay codigos de paises en nuestra data principal, que no figuran en nuestra tabla de paises
# entonces, vemos a decirle que si es nulo, muestre el codigo del pais de la tabla principal
data_final['Country Name, Abbreviation'] = data_final['Country Name, Abbreviation'].fillna(data_final['country'])


In [46]:
# Nota: hay un inconveniente con el país Namibia, Africa
# su codigo es NA, y me parece que se genera una confusion con NAN (nulo)
# para solucionarlo momentaneamente voy a cambiar a mano el valor a "Namibia"
data_final['Country Name, Abbreviation'] = data_final['Country Name, Abbreviation'].replace({'NA': 'Namibia'}, regex=True)

In [47]:
# Eliminamos las columnas que no nos sirven
data_final = data_final.drop(['country','ISO2-digit Alpha'],axis=1)

In [48]:
# Renombramos nuestra columna para que quede com 'country'
data_final = data_final.rename(columns={"Country Name, Abbreviation": "country"})

In [49]:
# Reordenamos
data_final = data_final[["datetime", "country", "state","city","shape","duration(minutes)","comments", "datePosted","latitude","longitude"]]

In [50]:
# Y comprobamos una ultima vez.. se ve bien
status(data_final)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,datetime,0,0.0,0,0.0,69433,datetime64[ns]
1,country,0,0.0,0,0.0,164,object
2,state,0,0.0,0,0.0,840,object
3,city,0,0.0,0,0.0,10676,object
4,shape,0,0.0,0,0.0,30,object
5,duration(minutes),0,0.0,0,0.0,533,float64
6,comments,0,0.0,0,0.0,79997,object
7,datePosted,0,0.0,0,0.0,317,datetime64[ns]
8,latitude,0,0.0,0,0.0,19907,float64
9,longitude,0,0.0,0,0.0,19454,float64


In [51]:
# Nota: hay punto y coma en el campo de comments. Cuando lo transformo a csv me lo toma como separador de columna
# a pesar de que le declaro que separe solo por comas.
# esto lo soluciona momentaneamente:
data_final['comments'] = data_final['comments'].replace({';': ' '}, regex=True)

In [52]:
#guardamos el set limpio
data_final.to_csv('clean Data UFO.csv', index=False, sep =',')
#quotechar="'"

<img src="https://i.pinimg.com/originals/a5/42/48/a5424852abc98df69ebd4b4e366e1b6f.gif">

Aca continuaremos el analisis: https://www.kaggle.com/leogenzano/ufo-sightings-2-parte-analisis-exploratorio/